In [ ]:
import pandas as pd
from sklearn.externals import joblib
import numpy as np

from auxiliaries import ColumnSelector
from auxiliaries import plot_confusion_matrix

##### Załadowanie danych

In [ ]:
data = joblib.load('../pickles/events_database.pkl')
google_places = joblib.load('../pickles/google_places_flatten.pkl')
events_tags = pd.read_csv('../data/events-tag_table_events_tags.csv')
events_tags_rel = pd.read_csv('../data/events-tag_table_events_tags_rel.csv')
organisers = pd.read_csv('../data/events-tag_table_organisers.csv')

##### Dowiązanie informacji o typie miejsca

In [ ]:
google_places.rename(columns={'place_name': 'place_name_std'}, inplace=True)
google_places['types'] = google_places['types'].astype(str)
data = data.merge(google_places[['place_name_std','types']].drop_duplicates(), left_on='place_name_std', right_on='place_name_std')

##### Dowiązanie tagów do wydarzeń

In [ ]:
fb_events = data[['facebook_id', 'name', 'place_name_std', 'description', 'street_std', 'types']].dropna()
fb_events['facebook_id'] = fb_events['facebook_id'].astype(int)
events_tagged = fb_events.merge(events_tags_rel, left_on='facebook_id', right_on='event_facebook_id').merge(events_tags, left_on='event_tag_id', right_on='id')

##### Budowa modelu do określania przestrzeni wydarzenia

In [ ]:
df = events_tagged[events_tagged['category'] == 'PRZESTRZEŃ WYDARZENIA'][['tagname','description','name','place_name_std', 'types']]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier


stop_words = ['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'ależ', 'ani', 'aż', 'bardziej', 'bardzo', 'bo', 'bowiem', 'by', 'byli', 'bynajmniej', 'być', 'był', 'była', 'było', 'były', 'będzie', 'będą', 'cali', 'cała', 'cały', 'ci', 'cię', 'ciebie', 'co', 'cokolwiek', 'coś', 'czasami', 'czasem', 'czemu', 'czy', 'czyli', 'daleko', 'dla', 'dlaczego', 'dlatego', 'do', 'dobrze', 'dokąd', 'dość', 'dużo', 'dwa', 'dwaj', 'dwie', 'dwoje', 'dziś', 'dzisiaj', 'gdy', 'gdyby', 'gdyż', 'gdzie', 'gdziekolwiek', 'gdzieś', 'i', 'ich', 'ile', 'im', 'inna', 'inne', 'inny', 'innych', 'iż', 'ja', 'ją', 'jak', 'jaka', 'jakaś', 'jakby', 'jaki', 'jakichś', 'jakie', 'jakiś', 'jakiż', 'jakkolwiek', 'jako', 'jakoś', 'je', 'jeden', 'jedna', 'jedno', 'jednak', 'jednakże', 'jego', 'jej', 'jemu', 'jest', 'jestem', 'jeszcze', 'jeśli', 'jeżeli', 'już', 'ją', 'każdy', 'kiedy', 'kilka', 'kimś', 'kto', 'ktokolwiek', 'ktoś', 'która', 'które', 'którego', 'której', 'który', 'których', 'którym', 'którzy', 'ku', 'lat', 'lecz', 'lub', 'ma', 'mają', 'mało', 'mam', 'mi', 'mimo', 'między', 'mną', 'mnie', 'mogą', 'moi', 'moim', 'moja', 'moje', 'może', 'możliwe', 'można', 'mój', 'mu', 'musi', 'my', 'na', 'nad', 'nam', 'nami', 'nas', 'nasi', 'nasz', 'nasza', 'nasze', 'naszego', 'naszych', 'natomiast', 'natychmiast', 'nawet', 'nią', 'nic', 'nich', 'nie', 'niech', 'niego', 'niej', 'niemu', 'nigdy', 'nim', 'nimi', 'niż', 'no', 'o', 'obok', 'od', 'około', 'on', 'ona', 'one', 'oni', 'ono', 'oraz', 'oto', 'owszem', 'pan', 'pana', 'pani', 'po', 'pod', 'podczas', 'pomimo', 'ponad', 'ponieważ', 'powinien', 'powinna', 'powinni', 'powinno', 'poza', 'prawie', 'przecież', 'przed', 'przede', 'przedtem', 'przez', 'przy', 'roku', 'również', 'sama', 'są', 'się', 'skąd', 'sobie', 'sobą', 'sposób', 'swoje', 'ta', 'tak', 'taka', 'taki', 'takie', 'także', 'tam', 'te', 'tego', 'tej', 'temu', 'ten', 'teraz', 'też', 'to', 'tobą', 'tobie', 'toteż', 'trzeba', 'tu', 'tutaj', 'twoi', 'twoim', 'twoja', 'twoje', 'twym', 'twój', 'ty', 'tych', 'tylko', 'tym', 'u', 'w', 'wam', 'wami', 'was', 'wasz', 'wasza', 'wasze', 'we', 'według', 'wiele', 'wielu', 'więc', 'więcej', 'wszyscy', 'wszystkich', 'wszystkie', 'wszystkim', 'wszystko', 'wtedy', 'wy', 'właśnie', 'z', 'za', 'zapewne', 'zawsze', 'ze', 'zł', 'znowu', 'znów', 'został', 'żaden', 'żadna', 'żadne', 'żadnych', 'że', 'żeby']

X = df[['description','name','place_name_std','types']]
y = df['tagname']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify = y)

alpha=0.1
max_features=500

clf_desc = Pipeline([('sel', ColumnSelector('description')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])
clf_name = Pipeline([('sel', ColumnSelector('name')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])
clf_place = Pipeline([('sel', ColumnSelector('place_name_std')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])
clf_types = Pipeline([('sel', ColumnSelector('types')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])

eclf = VotingClassifier(estimators=[('desc', clf_desc), 
                                    ('name', clf_name), 
                                    ('place', clf_place),
                                    ('types', clf_types)],
                                    voting='soft',
                                    weights=[1,1,1,1])
eclf = eclf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
%matplotlib inline

predicted = eclf.predict(X_test)

conf_arr = confusion_matrix(y_test, predicted, labels=eclf.classes_.tolist())

plot_confusion_matrix(conf_arr, classes=eclf.classes_.tolist(),
                      title='Confusion matrix, without normalization', normalize=True,)

In [ ]:
eclf_1 = eclf.fit(X,y)

##### Budowa modelu do określania typu wydarzeń

In [ ]:
df = events_tagged[events_tagged['category'] == 'TYP WYDARZENIA'][['tagname','description','name','place_name_std', 'types']]
df = df.groupby('tagname').filter(lambda x: len(x) > 1) 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier


stop_words = ['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'ależ', 'ani', 'aż', 'bardziej', 'bardzo', 'bo', 'bowiem', 'by', 'byli', 'bynajmniej', 'być', 'był', 'była', 'było', 'były', 'będzie', 'będą', 'cali', 'cała', 'cały', 'ci', 'cię', 'ciebie', 'co', 'cokolwiek', 'coś', 'czasami', 'czasem', 'czemu', 'czy', 'czyli', 'daleko', 'dla', 'dlaczego', 'dlatego', 'do', 'dobrze', 'dokąd', 'dość', 'dużo', 'dwa', 'dwaj', 'dwie', 'dwoje', 'dziś', 'dzisiaj', 'gdy', 'gdyby', 'gdyż', 'gdzie', 'gdziekolwiek', 'gdzieś', 'i', 'ich', 'ile', 'im', 'inna', 'inne', 'inny', 'innych', 'iż', 'ja', 'ją', 'jak', 'jaka', 'jakaś', 'jakby', 'jaki', 'jakichś', 'jakie', 'jakiś', 'jakiż', 'jakkolwiek', 'jako', 'jakoś', 'je', 'jeden', 'jedna', 'jedno', 'jednak', 'jednakże', 'jego', 'jej', 'jemu', 'jest', 'jestem', 'jeszcze', 'jeśli', 'jeżeli', 'już', 'ją', 'każdy', 'kiedy', 'kilka', 'kimś', 'kto', 'ktokolwiek', 'ktoś', 'która', 'które', 'którego', 'której', 'który', 'których', 'którym', 'którzy', 'ku', 'lat', 'lecz', 'lub', 'ma', 'mają', 'mało', 'mam', 'mi', 'mimo', 'między', 'mną', 'mnie', 'mogą', 'moi', 'moim', 'moja', 'moje', 'może', 'możliwe', 'można', 'mój', 'mu', 'musi', 'my', 'na', 'nad', 'nam', 'nami', 'nas', 'nasi', 'nasz', 'nasza', 'nasze', 'naszego', 'naszych', 'natomiast', 'natychmiast', 'nawet', 'nią', 'nic', 'nich', 'nie', 'niech', 'niego', 'niej', 'niemu', 'nigdy', 'nim', 'nimi', 'niż', 'no', 'o', 'obok', 'od', 'około', 'on', 'ona', 'one', 'oni', 'ono', 'oraz', 'oto', 'owszem', 'pan', 'pana', 'pani', 'po', 'pod', 'podczas', 'pomimo', 'ponad', 'ponieważ', 'powinien', 'powinna', 'powinni', 'powinno', 'poza', 'prawie', 'przecież', 'przed', 'przede', 'przedtem', 'przez', 'przy', 'roku', 'również', 'sama', 'są', 'się', 'skąd', 'sobie', 'sobą', 'sposób', 'swoje', 'ta', 'tak', 'taka', 'taki', 'takie', 'także', 'tam', 'te', 'tego', 'tej', 'temu', 'ten', 'teraz', 'też', 'to', 'tobą', 'tobie', 'toteż', 'trzeba', 'tu', 'tutaj', 'twoi', 'twoim', 'twoja', 'twoje', 'twym', 'twój', 'ty', 'tych', 'tylko', 'tym', 'u', 'w', 'wam', 'wami', 'was', 'wasz', 'wasza', 'wasze', 'we', 'według', 'wiele', 'wielu', 'więc', 'więcej', 'wszyscy', 'wszystkich', 'wszystkie', 'wszystkim', 'wszystko', 'wtedy', 'wy', 'właśnie', 'z', 'za', 'zapewne', 'zawsze', 'ze', 'zł', 'znowu', 'znów', 'został', 'żaden', 'żadna', 'żadne', 'żadnych', 'że', 'żeby']


df2 = df
X = df2[['description','name','place_name_std', 'types']]
y = df2['tagname']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify = y)

alpha=0.1
max_features=500

clf_desc = Pipeline([('sel', ColumnSelector('description')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])
clf_name = Pipeline([('sel', ColumnSelector('name')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])
clf_place = Pipeline([('sel', ColumnSelector('place_name_std')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])
clf_types = Pipeline([('sel', ColumnSelector('types')),
                     ('tfidf', TfidfVectorizer(stop_words=stop_words, use_idf=True, ngram_range=(1,1), max_features=max_features)),
                     ('clf', MultinomialNB(alpha=alpha, fit_prior=False))])

eclf = VotingClassifier(estimators=[('desc', clf_desc), 
                                    ('name', clf_name), 
                                    ('place', clf_place),
                                    ('types', clf_types)],
                                    voting='soft',
                                    weights=[1,1,1,1])

eclf = eclf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
%matplotlib inline

predicted = eclf.predict(X_test)

conf_arr = confusion_matrix(y_test, predicted, labels=eclf.classes_.tolist())

plot_confusion_matrix(conf_arr, classes=eclf.classes_.tolist(),
                      title='Confusion matrix, without normalization', normalize=True, size=14)

In [ ]:
eclf_2 = eclf.fit(X,y)

##### Predykcje dla wszystkich wydarzeń

In [ ]:
name = data.name\
.mask(pd.isnull, data.name_ss)\
.mask(pd.isnull, data.name_cjg)\
.mask(pd.isnull, data.name_sk)\
.mask(pd.isnull, data.name_um)

place_name = data.place_name\
.mask(pd.isnull, data.place_name_ss)\
.mask(pd.isnull, data.place_name_cjg)\
.mask(pd.isnull, data.place_name_sk)\
.mask(pd.isnull, data.place_name_um)

description = data.description\
.mask(pd.isnull, data.description_ss)\
.mask(pd.isnull, data.description_cjg)\
.mask(pd.isnull, data.description_sk)\
.mask(pd.isnull, data.description_um)

types = data['types'].tolist()

In [ ]:
event_space = eclf_1.predict(pd.DataFrame({'description': description, 'name': name, 'place_name_std': place_name, 'types': types}))
event_type = eclf_2.predict(pd.DataFrame({'description': description, 'name': name, 'place_name_std': place_name, 'types': types}))

In [ ]:
data['event_space'] = event_space
data['event_type'] = event_type

##### Serializacja modeli

In [ ]:
joblib.dump(eclf_1, '../models/events_space.pkl')
joblib.dump(eclf_2, '../models/events_type.pkl')

##### Zapisanie do pliku

In [ ]:
data.to_csv('../output/events_database.csv')
joblib.dump(data, '../pickles/events_database.pkl')